In [ ]:
#importa as bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import folium 
!pip install prettymatrix
import prettymatrix

In [ ]:
#LISTA PARAS AS BASES DE MESES PARES EM 2018

lista_bases = []
lista_bases.append('/content/drive/MyDrive/MAE0339/CitiByke/201802-citibike-tripdata.csv')
lista_bases.append('/content/drive/MyDrive/MAE0339/CitiByke/201804-citibike-tripdata.csv')
lista_bases.append('/content/drive/MyDrive/MAE0339/CitiByke/201806-citibike-tripdata.csv')
lista_bases.append('/content/drive/MyDrive/MAE0339/CitiByke/201808-citibike-tripdata.csv')
lista_bases.append('/content/drive/MyDrive/MAE0339/CitiByke/201810-citibike-tripdata.csv')
lista_bases.append('/content/drive/MyDrive/MAE0339/CitiByke/201812-citibike-tripdata.csv')

#LISTA PARA ARMAZENAR CLUSTERS
lista_clusters = []

#PARA CADA BASE
for base in lista_bases:

  #IMPORTA BASE DE DADOS
  data = pd.read_csv(base)

  #ADICIONA COLUNA PRA NUMERO DE PARTIDAS
  data.insert(3,'number_partidas',0,True)

  #ADICIONA COLUNA PARA NUMERO DE DESTINOS
  data.insert(4,'number_destinos',0,True)


  #NUMERO DE PARTIDAS DE CADA ESTAÇÃO DE PARTIDA
  data_mais_partidas = data[['start station name','number_partidas']]

  #CONTA E COLOCA EM NUMBER DE PARTIDAS
  data_mais_partidas = data_mais_partidas.groupby(by = ['start station name'], as_index = False).count()

  #ORDENA EM ORDEM  DECRESCENTE E RESETA INDEX
  data_mais_partidas.sort_values(by = 'number_partidas',ascending= False,inplace = True)
  data_mais_partidas.reset_index(inplace = True)
  
  print(f"#################################### MÊS {base[-24:-22]} #######################################")
  
  #ESTAÇÃO COM MAIS PARTIDAS
  print('ESTAÇÃO COM MAIS PARTIDAS DA BASE DE DADOS:\n',data_mais_partidas['start station name'][0])

  #DATA DAS ESTAÇÕES COM PARTIDA NA MAIS TOP DE PARTIDA
  data_mais_chegada = data[data['start station name'] == data_mais_partidas['start station name'][0]][['end station name','number_destinos']]

  #AGRUPA POR END STATION E CONTA E JOGA EM NUMBER DESTINOS
  data_mais_chegada = data_mais_chegada.groupby(by = ['end station name'],as_index = False).count()

  #ORDENA VALORES EM ORDEM DECRESCENTE E RESETA O INDEX
  data_mais_chegada.sort_values(by = 'number_destinos', ascending = False, inplace = True)
  data_mais_chegada.reset_index(inplace = True)


  #LISTA DAS 10 BOAS ESTAÇÕES PRO PRIMEIRO CLUSTER
  lista_estacoes = []

  #APPENDA A COM MAIS PARTIDAS DA BASE
  lista_estacoes.append(data_mais_partidas['start station name'][0])

  #APPENDA TODAS AS ESTAÇÕES TOP DE CHEGADAS A PARTIR DA TOP DE PARTIDAS

  print('--------------------------------------------------------------------------')
  print(f'ESTAÇÕES MAIS POPULARES DE CHEGADA:')

  if len(data_mais_chegada['end station name']) >= 10:

    for i in range(10):

      if data_mais_chegada['end station name'][i] != data_mais_partidas['start station name'][0]:

        lista_estacoes.append(data_mais_chegada['end station name'][i])
        print(data_mais_chegada['end station name'][i])

      #PARA QUANDO COMPLETAR 10 ELEMENTOS NA LISTA
      if len(lista_estacoes) == 10: break
  
  else:

    tamanho_possivel = len(data_mais_chegada['end station name'])

    for i in range(tamanho_possivel):

      if data_mais_chegada['end station name'][i] != data_mais_partidas['start station name'][0]:

        lista_estacoes.append(data_mais_chegada['end station name'][i])
        print(data_mais_chegada['end station name'][i])

      if len(lista_estacoes) == tamanho_possivel: break

  lista_clusters.append(lista_estacoes)

  #DATA DAS VIAGENS QUE TEM CHEGADA E PARTIDA PERTECENTES AO PRIMEIRO CLUSTER
  data_das_10 = data[data['start station name'].isin(lista_estacoes) & data['end station name'].isin(lista_estacoes)]

  #matriz de transição
  matriz_transicao = [0]*len(lista_estacoes)

  for j in range(len(lista_estacoes)):

    #TOTAL DE PARTIDAS NUMA LINHA
    total_partidas_em_j = len(data_das_10[data_das_10['start station name'] == lista_estacoes[j]])

    #PARTIDA NESSA LINHA E CHEGADA NA COLUNA i
    for i in range(len(lista_estacoes)):
      chegadas_em_i_partindo_em_j = len(data_das_10[data_das_10['start station name']== lista_estacoes[j]][data_das_10['end station name'] == lista_estacoes[i]])
      
      #PROBABILIDADE
      razao =  chegadas_em_i_partindo_em_j/total_partidas_em_j

      #SE FOR O PRIMEIRO DA LINHA CRIA UMA LISTA PRA ESSA LINHA E APPENDA O VALOR

      if i== 0: 
        matriz_transicao[j] = []
        matriz_transicao[j].append(round(razao,5))

      #APPENDA NA LISTA JÁ CRIADA
      else:
        matriz_transicao[j].append(round(razao,5))

  print('-----------------------------------------------------------------------')
  print('Matriz de transição para o primeiro cluster:')
  print(prettymatrix.matrix_to_string(np.array(matriz_transicao)))
  print('-----------------------------------------------------------------------')


  #CONJUNTO DE ESTAÇÕES DE PARTIDA QUE TEM ALGUMA RELAÇÃO COM O PRIMEIRO CLUSTER
  boas_ida = set(data[data['start station name'].isin(lista_estacoes) | data['end station name'].isin(lista_estacoes)]['start station name'].unique())

  #CONJUNTO DE ESTAÇÕES DE CHEGADA QUE TEM ALGUMA RELAÇÃO COM O SEGUNDO CLUSTER
  boas_chegada = set(data[data['start station name'].isin(lista_estacoes) | data['end station name'].isin(lista_estacoes)]['end station name'].unique())


  #IMPORTA BASE NOVAMENTE 
  data = pd.read_csv(base)

  #DATA DOS TRAJETOS QUE CUJA NEM ESTAÇÃO DE CHEGADA NEM ESTAÇÃO DE PARTIDA TEM RELAÇÃO COM PRIMEIRO CLUSTER
  data_ruins = data[~data['start station name'].isin(list(boas_ida)) & ~data['end station name'].isin(list(boas_chegada))]

  #insere coluna number_partidas
  data_ruins.insert(3,'number_partidas',0,True)

  #cria coluna number_destinos
  data_ruins.insert(4,'number_destinos',0,True)

  #classica startin station por number_partidas nela
  data_ruins_mais_partidas = data_ruins[['start station name','number_partidas']].groupby(by = ['start station name'], as_index = False).count().sort_values(by = 'number_partidas',ascending= False).reset_index()

  #data em ordem crescente das estações de chegada a partir da mais top de partida
  data_ruins_mais_chegada = data_ruins[data_ruins['start station name'] == data_ruins_mais_partidas['start station name'][0]][['end station name','number_destinos']].groupby(by = ['end station name'],as_index = False).count().sort_values(by = 'number_destinos', ascending = False).reset_index()

  #LISTA 10 BOAS ESTAÇÕES PRO SEGUNDO CLUSTER
  lista_estacoes = []

  #appenda os elementos
  lista_estacoes.append(data_ruins_mais_partidas['start station name'][0])

  print('Estação não correlata com primeiro cluster com mais partidas:\n',data_ruins_mais_partidas['start station name'][0])
  print('---------------------------------------------------------------------------------------------------------------')
  print('Estações mais populares de chegada com partida na anterior:')
  
  for i in range(len(lista_estacoes)):

    #se bater com mais top de partida não coloca na lista
    if data_ruins_mais_chegada['end station name'][i] != data_ruins_mais_partidas['start station name'][0]:
      lista_estacoes.append(data_ruins_mais_chegada['end station name'][i])
      print(data_ruins_mais_chegada['end station name'][i])

    #para quando completar 10 elementos na lista
    if len(lista_estacoes) == 10: break

  lista_clusters.append(lista_estacoes)

  print('--------------------------------------------------------------------------------------------------------------')
  #DATA CUJAS VIAGENS TEM PARTIDA E CHEGADA NAS ESTAÇÕES DO SEGUNDO CLUSTER
  data_ruins_das_10 = data[data['start station name'].isin(lista_estacoes) & data['end station name'].isin(lista_estacoes)]

  #MATRIZ TRANSIÇÃO SEGUNDO CLUSTER
  matriz_transicao = [0]*len(lista_estacoes)

  for j in range(len(lista_estacoes)):

    #TOTAL DE PARTIDAS NUMA LINHA
    total_partidas_em_j = len(data_ruins_das_10[data_ruins_das_10['start station name']== lista_estacoes[j]])

    #PARTIDAS NESSA LINHA E CHEGADA NA COLUNA i
    
    for i in range(len(lista_estacoes)):
      chegadas_em_i_partindo_em_j = len(data_ruins_das_10[data_ruins_das_10['start station name']== lista_estacoes[j]][data_ruins_das_10['end station name'] == lista_estacoes[i]])
      
      #SE A PROBABILIDADE NA LINHA FOR NULA APPENDA 0
      
      if i == 0 and total_partidas_em_j == 0:
        matriz_transicao[j] = []
        matriz_transicao[j].append(0)

      if i >= 1 and total_partidas_em_j == 0:
        matriz_transicao[j].append(0)

      #CASO CONTRÁRIO FAZ O PROCESSO IGUAL:

      if total_partidas_em_j != 0:

        razao =  chegadas_em_i_partindo_em_j/total_partidas_em_j

        #se for o primeiro da coluna cria uma lista pra essa linha e appenda
        if i==0: 
          matriz_transicao[j] = []
          matriz_transicao[j].append(round(razao,5))

        #appenda na lista já criada
        else:
          matriz_transicao[j].append(round(razao,5))

  print('Matriz de transição para o segundo cluster pro mes',base[-24:-22])
  print(prettymatrix.matrix_to_string(np.array(matriz_transicao)))





#################################### MÊS 02 #######################################
ESTAÇÃO COM MAIS PARTIDAS DA BASE DE DADOS:
 Pershing Square North
--------------------------------------------------------------------------
ESTAÇÕES MAIS POPULARES DE CHEGADA:
E 24 St & Park Ave S
Broadway & W 32 St
W 33 St & 7 Ave
W 41 St & 8 Ave
E 32 St & Park Ave
E 30 St & Park Ave S
Broadway & E 22 St
8 Ave & W 33 St
5 Ave & E 29 St


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:101: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


-----------------------------------------------------------------------
Matriz de transição para o primeiro cluster:
┌                                                                                 ┐
│ 0.01489 0.14833 0.14779 0.14673 0.11855 0.09569 0.08825 0.08187 0.08134 0.07656 │
│ 0.40041 0.05394 0.0249  0.05602 0.13693 0.12033 0.04149 0.07676 0.06432 0.0249  │
│ 0.23474 0.04225 0.03052 0.03286 0.10563 0.23239 0.07512 0.15728 0.0446  0.0446  │
│ 0.35783 0.05029 0.08124 0.04062 0.03288 0.15861 0.06576 0.09091 0.02901 0.09284 │
│ 0.34409 0.12186 0.08244 0.05197 0.0448  0.03763 0.1362  0.06989 0.06093 0.05018 │
│ 0.25581 0.07752 0.17829 0.16667 0.07558 0.0407  0.01744 0.06008 0.05233 0.07558 │
│ 0.49045 0.03397 0.06157 0.09342 0.11677 0.01274 0.0552  0.0828  0.03185 0.02123 │
│ 0.44037 0.0344  0.02294 0.0344  0.07569 0.07339 0.12615 0.08486 0.06422 0.04358 │
│ 0.37687 0.05037 0.05224 0.04104 0.19403 0.11381 0.02052 0.0597  0.05224 0.03918 │
│ 0.22687 0.05286 0.07048 0.09692 0.13656 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:184: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


#################################### MÊS 04 #######################################
ESTAÇÃO COM MAIS PARTIDAS DA BASE DE DADOS:
 Pershing Square North
--------------------------------------------------------------------------
ESTAÇÕES MAIS POPULARES DE CHEGADA:
Broadway & W 32 St
W 33 St & 7 Ave
E 24 St & Park Ave S
W 41 St & 8 Ave
E 30 St & Park Ave S
Broadway & E 22 St
E 32 St & Park Ave
E 33 St & 2 Ave
8 Ave & W 33 St
-----------------------------------------------------------------------
Matriz de transição para o primeiro cluster:
┌                                                                                 ┐
│ 0.02413 0.17113 0.15489 0.13778 0.12681 0.08205 0.08118 0.07723 0.07372 0.07108 │
│ 0.21988 0.05422 0.05873 0.03464 0.08886 0.06024 0.15663 0.18675 0.08735 0.05271 │
│ 0.26893 0.13838 0.03133 0.04047 0.02742 0.06658 0.07833 0.16841 0.14752 0.03264 │
│ 0.3742  0.05255 0.06051 0.0828  0.1258  0.03025 0.06369 0.11306 0.04299 0.05414 │
│ 0.28921 0.07338 0.07194 0.09928 0.06

##item C

In [ ]:
#importa bilbio folium
import folium 

k = lista_clusters[0]

#cria um mapa
m = folium.Map(location = [40.730610,-73.935242],zoom_start= 13,width = '100%')

for j in range(10):

    for w in range(len(data['start station name'])):

      if k[j] == data['start station name'][w]:

        folium.Marker([float(data['start station latitude'][j]),float(data['start station longitude'][j])], popup="<b>Timberline Lodge</b>", tooltip=tooltip
,icon=folium.Icon(color='green')).add_to(m)

m


                              